# 2487-2223 Machine Learning Assignment 1



## Question 1 (25 points) - Zestimate this House

Purchasing a house is a very big decision for most of us. Companies such as Zillows collected tons of data regarding the listing and sold price of American houses and build the predictive model, named *Zestimate*. You are expected to build a model similar as Zestimate to predict house price in Boston. 

![zestimate](https://i0.wp.com/www.housesoldeasy.com/wp-content/uploads/Screen-Shot-2016-08-15-at-7.22.09-PM.png?resize=300%2C258&ssl=1)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, LogisticRegression, LogisticRegressionCV, Ridge, Lasso
from sklearn.metrics import mean_squared_error, classification_report, precision_score, recall_score, confusion_matrix, precision_recall_curve
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

In [3]:
### DON'T MODIFY - LOAD DATA ### 

data_url = "http://lib.stat.cmu.edu/datasets/boston" 
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]]) # FEATURES 
y = raw_df.values[1::2, 2] # TARGET VARIABLE
assert X.shape[0] == y.shape[0], 'Mismatch in number of examples.'
print('Data loaded correctly.')
print('Features: X. Target variable (price): y.')
print('X shape: ',X.shape, 'y shape: ', y.shape)
### END ###

Data loaded correctly.
Features: X. Target variable (price): y.
X shape:  (506, 13) y shape:  (506,)


In [4]:
X

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

#### Question 1.1 (5 points) 
Create train and test set, each contains 80% and 20% of the dataset, respectively, using *train_test_split* function in scikit-learn. Train a linear model on the train set and evaluate on the test set, report the training error and test error, respectively (as mean squared error).

In [5]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a linear model on the train set
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model on the train and test sets
train_pred = model.predict(X_train)
train_mse = mean_squared_error(y_train, train_pred)
test_pred = model.predict(X_test)
test_mse = mean_squared_error(y_test, test_pred)

# Print the training and test errors
print('Training MSE: ' + str(round(train_mse, 2)))
print('Test MSE: ' + str(round((test_mse), 2)))

Training MSE: 21.64
Test MSE: 24.29


**Analysis**
- Test error > training error -> model is overfitting
- Low errors -> linear model is performing well on this dataset

#### Question 1.2 (5 points)

Perform a 10-fold cross-validation on the whole data set. Show the averaged mean sqaured error on both train and test set at each fold. Explain your findings.

In [6]:
# Define 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Train a linear model and perform 10-fold cross-validation
model = LinearRegression()
train_scores = []
test_scores = []
for train_idx, test_idx in kfold.split(X, y):
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_scores.append(mean_squared_error(y_train, train_pred))
    test_scores.append(mean_squared_error(y_test, test_pred))

# Calculate and print the average mean squared error on both train and test set at each fold
for i in range(10):
    print('Fold', i+1, 'Train MSE:', train_scores[i], 'Test MSE:', test_scores[i])
print('Average Train MSE:', np.mean(train_scores))
print('Average Test MSE:', np.mean(test_scores))

Fold 1 Train MSE: 22.7375901544866 Test MSE: 14.99585287658265
Fold 2 Train MSE: 20.85282114086763 Test MSE: 32.80452455251558
Fold 3 Train MSE: 22.523708930639955 Test MSE: 17.599677176293966
Fold 4 Train MSE: 21.77705689266885 Test MSE: 23.36882696420213
Fold 5 Train MSE: 20.47610322792512 Test MSE: 35.1525526337427
Fold 6 Train MSE: 22.262543646395933 Test MSE: 19.155888640230433
Fold 7 Train MSE: 21.696285739040018 Test MSE: 24.140705056979968
Fold 8 Train MSE: 22.206382674389694 Test MSE: 19.5477124442254
Fold 9 Train MSE: 22.18801366254756 Test MSE: 20.26815402662742
Fold 10 Train MSE: 21.465363892478834 Test MSE: 26.60813570390883
Average Train MSE: 21.818586996144017
Average Test MSE: 23.36420300753091


**Analysis**

We can see that the mean squared error on both the train and test set varies across the 10 folds of the cross-validation. The train MSE ranges from 20.48 to 22.74, while the test MSE ranges from 15 to 34.15. The average train MSE across the 10 folds is 21.82, which is similar to the training error we obtained in the previous question. The average test MSE across the 10 folds is 23.36, which is also similar to the test error we obtained in the previous question. This indicates that the linear model is still overfitting the training set to some extent, but the cross-validation results suggest that the overfitting is not as severe as we initially thought. Overall, the results suggest that the linear model has moderate predictive power on the Boston Housing dataset.

In [7]:
# Testar cross_validate
# Define the model
model = LinearRegression()

# Define the cross-validation method
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Evaluate the model using cross-validation
train_scores = -cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv)
test_scores = -cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv)

# Print the train and test scores for each fold
for i in range(10):
    print('Fold ' + str(i+1) + ': Train MSE: ' + str(round(train_scores[i], 2)) + ', Test MSE: ' + str(round(test_scores[i], 2)))

# Print the averaged train and test scores over all folds
print('Average Train MSE: ' + str(round(train_scores.mean(), 2)))
print('Average Test MSE: ' + str(round(test_scores.mean(), 2)))

Fold 1: Train MSE: 15.0, Test MSE: 15.0
Fold 2: Train MSE: 32.8, Test MSE: 32.8
Fold 3: Train MSE: 17.6, Test MSE: 17.6
Fold 4: Train MSE: 23.37, Test MSE: 23.37
Fold 5: Train MSE: 35.15, Test MSE: 35.15
Fold 6: Train MSE: 19.16, Test MSE: 19.16
Fold 7: Train MSE: 24.14, Test MSE: 24.14
Fold 8: Train MSE: 19.55, Test MSE: 19.55
Fold 9: Train MSE: 20.27, Test MSE: 20.27
Fold 10: Train MSE: 26.61, Test MSE: 26.61
Average Train MSE: 23.36
Average Test MSE: 23.36


#### Question 1.3 (5 points) 
 
Add 2-degree squared polynomial features (with no interactions) and perform 10-fold cross-validation on the whole data set. Show the mean sqaured error on both train and test set at each fold. Explain your findings.

Hint: you may use sklearn.preprocessing.PolynomialFeatures and check how it produces the polynomial features with/without interaction terms.

In [8]:
# Add 2-degree squared polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

# Define 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Train a linear model with polynomial features and perform 10-fold cross-validation
model = LinearRegression()
train_scores = []
test_scores = []
for train_idx, test_idx in kfold.split(X_poly, y):
    X_train, y_train = X_poly[train_idx], y[train_idx]
    X_test, y_test = X_poly[test_idx], y[test_idx]
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    train_scores.append(mean_squared_error(y_train, train_pred))
    test_scores.append(mean_squared_error(y_test, test_pred))

# Calculate and print the average mean squared error on both train and test set at each fold
for i in range(10):
    print('Fold', i+1, 'Train MSE:', train_scores[i], 'Test MSE:', test_scores[i])
print('Average Train MSE:', np.mean(train_scores))
print('Average Test MSE:', np.mean(test_scores))


Fold 1 Train MSE: 5.945580064091453 Test MSE: 8.756633514670874
Fold 2 Train MSE: 5.165618833342352 Test MSE: 20.450915594154523
Fold 3 Train MSE: 5.848722041597204 Test MSE: 13.296157702197098
Fold 4 Train MSE: 5.678288605419227 Test MSE: 15.529903996120405
Fold 5 Train MSE: 5.574123289586335 Test MSE: 13.882716128932193
Fold 6 Train MSE: 6.020009539191569 Test MSE: 21.66779982244787
Fold 7 Train MSE: 5.860454243811597 Test MSE: 9.74468351182666
Fold 8 Train MSE: 6.0108147811452675 Test MSE: 7.218812707841904
Fold 9 Train MSE: 6.0863294304804505 Test MSE: 7.326348286436629
Fold 10 Train MSE: 5.7209094813251555 Test MSE: 12.867541906597461
Average Train MSE: 5.791085030999062
Average Test MSE: 13.074151317122562


**Analysis**

We can see that adding 2-degree squared polynomial features to the dataset improves the performance of the linear model. The train MSE ranges from 5.17 to 6.09, while the test MSE ranges from 7.22 to 21.67. The average train MSE across the 10 folds is 5.72, which is lower than the training error we obtained in the previous question. The average test MSE across the 10 folds is 13.07, which is also lower.

#### Question 1.4 (10 points)

Perform cross-validation using ridge regression and lasso regression on different feature combinations (linear features vs. polynomial features obtained earlier respectively. Explain which method works better in this case. Check the coefficients and explain the differences between ridge regression and lasso regression.

In [9]:
# Define 10-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# Add polynomial features to linear data
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

# Train a linear model with ridge regression and perform 10-fold cross-validation
ridge = Ridge(alpha=1.0) #[0.01, 0.1, 1, 10, 100]
ridge.fit(X, y)
ridge_test_scores = -cross_val_score(ridge, X, y, cv=kfold, scoring='neg_mean_squared_error') # train = -ridge.score(X, y)
print('Linear Features with Ridge Regression')
print('Test MSE:', np.mean(ridge_test_scores))
print('Coef of Linear Features with Ridge Regression:', ridge.coef_, '\n')

# Perform cross-validation on polynomial features using Ridge regression
ridge = Ridge(alpha=1.0)
ridge.fit(X_poly, y)
poly_ridge_test_scores = -cross_val_score(ridge, X_poly, y, cv=kfold, scoring='neg_mean_squared_error')
print('Poly Features with Ridge Regression')
print('Test MSE:', np.mean(poly_ridge_test_scores))
print('Coef of Poly Features with Ridge Regression:', ridge.coef_, '\n')

# Train a linear model with lasso regression and perform 10-fold cross-validation
lasso = Lasso(alpha=1.0)
lasso.fit(X, y)
lasso_test_scores = -cross_val_score(lasso, X, y, cv=kfold, scoring='neg_mean_squared_error')
print('Linear Features with Lasso Regression')
print('Test MSE:', np.mean(lasso_test_scores))
print('Coef of Linear Features with Lasso Regression:', lasso.coef_, '\n')

# Perform cross-validation on polynomial features using Lasso regression
lasso = Lasso(alpha=1.0)
lasso.fit(X_poly, y)
poly_lasso_test_scores = -cross_val_score(lasso, X_poly, y, cv=kfold, scoring='neg_mean_squared_error')
print('Poly Features with Lasso Regression')
print('Test MSE:', np.mean(poly_lasso_test_scores))
print('Coef of Poly Features with Lasso Regression:', lasso.coef_)

Linear Features with Ridge Regression
Test MSE: 23.55453050994863
Coef of Linear Features with Ridge Regression: [-1.04595278e-01  4.74432243e-02 -8.80467889e-03  2.55239322e+00
 -1.07770146e+01  3.85400020e+00 -5.41453810e-03 -1.37265353e+00
  2.90141589e-01 -1.29116463e-02 -8.76074394e-01  9.67327945e-03
 -5.33343225e-01] 

Poly Features with Ridge Regression
Test MSE: 14.430148406128884
Coef of Poly Features with Ridge Regression: [-1.10562932e+00 -2.89393780e-01 -2.23752565e+00  9.12521424e-01
  1.07129832e-01  4.72828819e+00  8.57778667e-01 -1.40989374e+00
  2.04860314e+00 -4.61157073e-02 -1.64531921e+00  1.20050806e-01
 -1.13488290e-01  1.95916639e-03  1.36127122e-01  3.43338914e-01
  3.00277719e+00 -8.99888402e-01  1.09657662e-01 -3.54182593e-03
 -6.58705406e-02  2.26736420e-01 -2.20769504e-02  1.95265866e-01
 -3.14998954e-04  1.82561742e-02  2.82404131e-05 -1.89439780e-03
 -5.01240048e-02 -1.50624424e-01  1.88717497e-02 -4.08798790e-04
 -7.80040639e-03 -4.68314381e-03  3.676759

**Analysis**

From the output, we can see that the polynomial features models perform much better when compared to the linear models. The polynomial feature model is has MSE of around 14, while the linear model is always above 23.

The main difference between Ridge regression and Lasso regression is the type of regularization term they use. Ridge regression uses L2 regularization, which adds a penalty term to the sum of squares of the coefficients of the independent variables. Lasso regression, on the other hand, uses L1 regularization, which adds a penalty term to the sum of absolute values of the coefficients of the independent variables.

One of the main consequences of this difference is that Ridge regression tends to produce solutions with small but non-zero coefficients for all independent variables, while Lasso regression tends to produce sparse solutions where some coefficients are exactly zero. This means that Lasso regression can be used for feature selection, as it effectively eliminates some independent variables from the model, while Ridge regression typically retains all independent variables in the model.

In our case, the method with lower MSE is the polynomial feature Lasso regression (14.2), but only by a small margin when compared to the polynomial feature Ridge regression (14.4).

## Question 2 (25 points) - Cancer Detection

Given a dataset with features that are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass, which describes characteristics of the cell nuclei present in the image, let's try to predict whether the patients are diagnosed as Malignant (M) or Benign (B).

In [10]:
# LOAD DATA 

from sklearn.datasets import load_breast_cancer
"""
DOCS:
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer
"""
X, y = load_breast_cancer(return_X_y=True)

#### Question 2.1 (5 points) 
Use logistic regression to train the dataset through cross-validation, report the score on train and test set, respectively. Explain your findings.

In [11]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a logistic regression model with L2 regularization and higher number of iterations
clf = LogisticRegression(penalty='l2', max_iter=3000, random_state=42)

# Train the model using 5-fold cross-validation on the train set
scores = cross_val_score(clf, X_train, y_train, cv=5)

# Report the mean score on the train set
print(f"Train score: {scores.mean():.3f}")

# Evaluate the model on the test set
clf.fit(X_train, y_train)
test_score = clf.score(X_test, y_test)

# Report the score on the test set
print(f"Test score: {test_score:.3f}")

Train score: 0.954
Test score: 0.956


**Analysis**

The logistic regression model achieved a high score on both the training set and the test set. The difference in score between the training set and the test set is also very small. This indicates that the model is not overfitting and can generalize well to unseen data. Overall, these findings suggest that the logistic regression model is a good fit for the breast cancer dataset and can accurately predict whether a tumor is benign or malignant.

#### Question 2.2 (5 points) 
By default, sklearn's logistic regression uses the L2 regularization. Now use the logistic regression without any regularzation to perform cross validation, report what do you find on train and test set.

In [12]:
# Initialize logistic regression without regularization
lr = LogisticRegression(penalty='none', solver='sag', max_iter=5000)

# Perform cross-validation with 5 folds
train_scores = cross_val_score(lr, X, y, cv=5)

# Fit the model on the train set
lr.fit(X_train, y_train)

# Evaluate the performance on the test set
test_score = lr.score(X_test, y_test)

# Report the average train score
print("Train score: {:.3f}".format(train_scores.mean()))

# Report the test score
print("Test score: {:.3f}".format(test_score))

Train score: 0.923
Test score: 0.965


#### Question 2.3 (15 points) 
Check how many Benign and Malignant cases in the dataset. What might be the problem if we use the default score of the logistic regression cross-validation? Now adjust the class weight of M and L and retrain the model again to bias toward Malignant, using the relative weight of M and L as 2:1. What about the relaive weight to be 5:1, or 10:1? Explain what you find.

Hint: you can use LogisticRegressionCV to combine LogisticRegression and cross-validation. 

In [13]:
benign_count = sum(y == 1)
malignant_count = sum(y == 0)

print(f"Benign count: {benign_count}")
print(f"Malignant count: {malignant_count}")

Benign count: 357
Malignant count: 212


The problem with using the default score of the logistic regression cross-validation is that it treats the two classes equally, which is not the case in this dataset. There are more benign cases than malignant cases. However, identifying malignant cases is the most important class to detect.

In [14]:
class_weights = [{0: 2, 1: 1}, {0: 5, 1: 1}, {0: 10, 1: 1}]
    
for c in class_weights:
    # Train the model with cross-validation and the class weight dictionary
    model = LogisticRegressionCV(class_weight=c, solver='sag', max_iter=5000)

    # Perform cross-validation with 5 folds
    train_scores = cross_val_score(model, X, y, cv=5)

    # Fit the model on the train set
    model.fit(X_train, y_train)

    # Evaluate the performance on the test set
    test_score = model.score(X_test, y_test)

    print(f"Class weight: {c}")
    # Report the average train score
    print("Train score: {:.3f}".format(train_scores.mean()))

    # Report the test score
    print("Test score: {:.3f}\n".format(test_score))

Class weight: {0: 2, 1: 1}
Train score: 0.914
Test score: 0.947

Class weight: {0: 5, 1: 1}
Train score: 0.893
Test score: 0.939

Class weight: {0: 10, 1: 1}
Train score: 0.854
Test score: 0.877



**Analysis**

Overall, the first configuration with a class weight of {0: 2, 1: 1} performs the best with the highest accuracy scores on both the train and test sets. This configuration strikes a balance between the two classes, with a slightly higher weight placed on malignant cance while still allowing the model to perform well on both classes. Therefore, this weight configuration is the best for this classification problem.

## Question 3 (50 points) - Call Me Maybe? 



![telemarketing](https://neilpatel.com/wp-content/uploads/2019/08/profissional-de-telemarketing-sorridente.jpeg)

Telemarketing is a method of direct marketing in which a salesperson solicits prospective customers to buy products or services over the phone. It has become one of the most widely used marketing campaign methods to engage with customers with product and service opportunity. We have collected real data from a Portuguese retail bank, from May 2008 to June 2013 with thousands of phone contacts. 




The current practice of many data teams is to build such propensity models and predict customer's probability to adopt the product and target them from the highest probability to the lowest probability. Note that telemarketing may incur some costs for contacting the customer, thus the success (i.e., the generated profit) of using machine learning model requries further inspection.  As the data scientist, you are asked to build a propensity model to evaluate the effectiveness of their telemarketing campaigns, i.e. whether the customer subscribed to the term deposit.  

**Telemarketing Dataset (bank.csv)**
All customers are contained in the file bank.csv. Each line of this file after the header row represents one customer of the Portuguese bank, and has the following format:

### bank client data:
- age (numeric)
- job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
- marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
- education (categorical: 'primary', 'secondary', 'tertiary')
- balance: amcount of bank account balance
- default: has credit in default? (categorical: 'no','yes','unknown')
- housing: has housing loan? (categorical: 'no','yes','unknown')
- loan: has personal loan? (categorical: 'no','yes','unknown')

### related with the last contact of the current campaign:
- contact: contact communication type (categorical: 'cellular','telephone', 'unknown')
- day: last contact day of month
- month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
- duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. 

### other attributes:
- campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
- pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; -1 means client was not previously contacted)
- previous: number of contacts performed before this campaign and for this client (numeric)
- poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
- y - has the client subscribed a term deposit? (binary: 'yes','no')


Answer the following questions using the provided dataset. You can write down intermediate results towards the final answers. If any model invovles random_state, set it to be 42.

In [15]:
bank = pd.read_csv('bank.csv', sep=';')
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [16]:
bank.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [17]:
bank.describe()

,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


In [18]:
bank.y.value_counts()

no     4000
yes     521
Name: y, dtype: int64

### Question 3.1 (15 points)

Split the data into 80% training set and 20% test set. **Build a pipeline to preprocess the indicated numerical features and categorical features separately**. For numerical features 'balance', 'campaign', standardize these features. For categorical features 'job', 'marital', 'education', 'default', transform them through one-hot encoding. For the numeric feature 'age', convert it into the quartile categorical variable and transform it through one-hot encoding. 

Train a Logistic regression model with L2 regularization using 5-fold cross validation (default hyperparameter) on the train set and show the accuracy, precision, recall on the train set. Explain whether the model is useful for the bank to identify the customer propensity.

In [19]:
X = bank.drop('y', axis=1)
y = bank['y'].apply(lambda x: 1 if x == 'yes' else 0)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Convert 'age' into quartile categorical variable
X_train['age'] = pd.qcut(X_train['age'], 4, labels=[0, 1, 2, 3])
X_test['age'] = pd.qcut(X_test['age'], 4, labels=[0, 1, 2, 3])

# Define the column transformer for the numerical and categorical features
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, ['balance', 'campaign']),
    ('cat', cat_transformer, ['job', 'marital', 'education', 'default', 'age'])
])

# Define the logistic regression model
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegressionCV(penalty='l2', cv=5))
])

# Fit the model on the training set
clf.fit(X_train, y_train)

# Predict on the training set and calculate accuracy, precision, and recall
y_pred = clf.predict(X_train)
accuracy = clf.score(X_train, y_train)
report = classification_report(y_train, y_pred, zero_division=0)
precision = precision_score(y_train, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_train, y_pred, average='weighted')

print(report)
print(f"Accuracy: {accuracy:.3f}")
print(f'Overall precision: {precision:.3f}')
print(f'Overall recall: {recall:.3f}')

              precision    recall  f1-score   support

           0       0.88      1.00      0.94      3193
           1       0.00      0.00      0.00       423

    accuracy                           0.88      3616
   macro avg       0.44      0.50      0.47      3616
weighted avg       0.78      0.88      0.83      3616

Accuracy: 0.883
Overall precision: 0.780
Overall recall: 0.883


**Analysis**

Based on the classification report, the model has an overall accuracy of 0.883, which indicates that it is able to predict the customer propensity with reasonable accuracy. However, it is important to note that the precision and recall for the positive class ('yes') are 0, which suggests that the model have difficulties identifying customers who are likely to accept the offer.

In addition, the dataset is imbalanced, with the negative class ('no') being much more prevalent than the positive class. This can lead to bias in the model's predictions and may result in an overly optimistic assessment of the model's performance.

Calculating the overall precision and recall we get the values 0.78 and 0.88, respectively. We observe the recall and accuracy have the same value.

Therefore, while the model shows promise, it may require further refinement and evaluation before it can be deemed useful for the bank to identify customer propensity.

### Question 3.2 (10 points)

Now add more features to the model to see if we can improve the performance (categorical features: 'housing', 'loan' and numerical features: 'day', 'duration'). Use the preprocess pipeline built previously to transform the data. Train a Logistic regression model with L1 regularization using 5-fold cross validation on the train set, by fine-tuning the hyperparameter alpha, i.e. the regularization strength from [0.001, 0.01, 0.1, 1]. Choose the correct score function that reflect the current data team's practice. Report the average score with the best hyperparameter. Does model performance improve, and if so, how?

Expalin whether all features are useful for making prediction and why. List top 5 features that contribute to the prediction the most. If not all features are useful, list those unuseful features.

In [21]:
# Features list
num_features = ['balance', 'campaign', 'day', 'duration']
cat_features = ['job', 'marital', 'education', 'default', 'age', 'housing', 'loan']

# Update the preprocessor with the new feature lists
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(), cat_features)
    ])

# Create a pipeline with L1 regularization
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(penalty='l1', solver='liblinear', class_weight='balanced', random_state=42))
])

# Define the hyperparameter search space
param_grid = {'classifier__C': [1 / 0.001, 1 / 0.01, 1 / 0.1, 1]}

# Perform a Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)

# Report the best hyperparameter and average score
best_hyperparameter = grid_search.best_params_['classifier__C']
best_score = grid_search.best_score_

print("Best hyperparameter:", best_hyperparameter)
print("Best average score:", best_score)


# Train the model with the best hyperparameter
best_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(penalty='l1', solver='liblinear', C=best_hyperparameter, class_weight='balanced', random_state=42))
])
best_pipeline.fit(X_train, y_train)

# Get feature importances
coef = best_pipeline.named_steps['classifier'].coef_[0]
onehot_columns = best_pipeline.named_steps['preprocessor'].transformers_[1][1].get_feature_names(cat_features)

# Combine feature names and coefficients
features = np.array(num_features + list(onehot_columns))
feature_importances = pd.DataFrame({'feature': features, 'coef': coef})

# Sort features by their absolute coefficient values
feature_importances['abs_coef'] = np.abs(feature_importances['coef'])
feature_importances = feature_importances.sort_values(by='abs_coef', ascending=False)

# Display the top 5 features
print("Top 5 features:\n", feature_importances.head(5))

# Display unuseful features (with a coefficient close to zero)
unuseful_features = feature_importances.sort_values(by='abs_coef', ascending=True) 
print("\nUnuseful features:\n", unuseful_features.head(5))

Best hyperparameter: 1
Best average score: 0.44540270977130164
Top 5 features:
              feature      coef  abs_coef
3           duration  1.355564  1.355564
32          loan_yes -1.001370  1.001370
6   job_entrepreneur -0.858314  0.858314
12       job_student  0.753710  0.753710
30       housing_yes -0.657190  0.657190

Unuseful features:
                 feature  coef  abs_coef
31              loan_no   0.0       0.0
18       marital_single   0.0       0.0
20  education_secondary   0.0       0.0
26                age_1   0.0       0.0
24          default_yes   0.0       0.0


**Analysis**

The best hyperparameter (C) found using Grid Search is 1, and the best average F1 score is 0.4458. 

Not all features are useful for making predictions. The output shows that some features have a coefficient of zero, indicating that they do not contribute to the model's predictions. These features are considered unuseful for making predictions. Specifically, the features "loan_no", "marital_single", "education_secondary", "age_1", and "default_yes" have coefficients of zero, indicating that they do not influence the model's predictions.

On the other hand, the top 5 features with the highest absolute coefficients are "duration", "loan_yes", "job_entrepreneur", "job_student", and "housing_yes". These features are considered the most useful for making predictions as they have the most significant impact on the model's outcome.

### Question 3.3 (10 points)

Now use the best model found in the cross-validation to predict the test set, show the obtained confusion matrix. Assume that targeting each customer would cost 10 euros and if the customer subscribe, the company would earn 50 euros. If we perform targeted telemarketing to all customers that are predicted to subscribe in the test set, what's the resulting profit?

In [22]:
# Predict the test set
y_pred_test = best_pipeline.predict(X_test)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)
print("Confusion Matrix:\n", conf_matrix)

# Get values from the confusion matrix
TN, FP, FN, TP = conf_matrix.ravel()

# Cost and profit settings
cost_per_customer = 10
profit_per_customer = 50

# Calculate profit
total_profit = (TP * profit_per_customer) - (TP + FP) * cost_per_customer
print("Total Profit:", total_profit)

Confusion Matrix:
 [[650 157]
 [ 30  68]]
Total Profit: 1150


### Question 3.4 (10 points)

Now adjust the decision threshold in order to optimize the obtained profit. What would be the resulting threshold and profit? Is the propensity model built based on the targeting predicted probability useful in terms of profit maximizing? Explain.

In [30]:
# Profit function
def calculate_profit(TP, FP):
    return (TP * profit_per_customer) - (TP + FP) * cost_per_customer

# Get the predicted probabilities of the positive class
y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]

# Calculate precision, recall, and thresholds using precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
thresholds = np.append(thresholds, 1)

# Calculate true positives and false positives for each threshold
true_positives = (recall * y_test.sum()).astype(int)
false_positives = ((1 - precision) * true_positives / precision).astype(int)

# Calculate the profit for each threshold
profits = [calculate_profit(tp, fp) for tp, fp in zip(true_positives, false_positives)]

# Find the optimal threshold and profit
optimal_index = np.argmax(profits)
optimal_threshold = thresholds[optimal_index]
optimal_profit = profits[optimal_index]

print("Optimal Threshold:", optimal_threshold)
print("Optimal Profit:", optimal_profit)

Optimal Threshold: 0.6353875394659373
Optimal Profit: 1300


**Analysis**

The code uses the best model from cross-validation to predict the probability of a customer subscribing. Precision, recall, and corresponding thresholds are then computed using the precision_recall_curve function. The goal is to find the optimal decision threshold that maximizes profit, achieved by calculating true positives and false positives for each threshold and computing the profit for each threshold using the calculate_profit function.

The output shows that the propensity model, based on the targeting predicted probability, is useful for maximizing profit. By adjusting the decision threshold, the company can optimize their telemarketing campaign to target customers more likely to subscribe and increase profit. The original threshold yielded a profit of 1,150 euros, but optimizing the threshold increased profit to 1,300 euros, demonstrating the usefulness of the propensity model in maximizing profit.

### Question 3.5 (5 points)

Now train a random forest model, with 10 decision trees and max_depth=2, what is the profit that can be achieved given the threshold that you identified earlier? Do you need to increase or decrese the threshold to maximize the profit using random forest model? Explain your result.

In [28]:
# Train the random forest model
rf_pipeline = RandomForestClassifier(n_estimators=10, max_depth=2, random_state=42)
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', rf_pipeline)
])
rf_pipeline.fit(X_train, y_train)

# Calculate profit with the threshold identified earlier
y_pred_proba_rf = rf_pipeline.predict_proba(X_test)[:, 1]
y_pred_rf = (y_pred_proba_rf > optimal_threshold).astype(int)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)

TN, FP, FN, TP = conf_matrix_rf.ravel()
profit_rf = (TP * profit_per_customer) - (TP + FP) * cost_per_customer

print("Profit with Random Forest:", profit_rf)

# Optimize the threshold for the random forest model
profits_rf = []

for threshold in thresholds:
    y_pred_threshold_rf = (y_pred_proba_rf > threshold).astype(int)
    conf_matrix_rf = confusion_matrix(y_test, y_pred_threshold_rf)

    TN, FP, FN, TP = conf_matrix_rf.ravel()
    profit_rf = (TP * profit_per_customer) - (TP + FP) * cost_per_customer
    profits_rf.append(profit_rf)

max_profit_index_rf = np.argmax(profits_rf)
optimal_threshold_rf = thresholds[max_profit_index_rf]
max_profit_rf = profits_rf[max_profit_index_rf]

print("Optimal Threshold for Random Forest:", optimal_threshold_rf)
print("Maximized Profit with Random Forest:", max_profit_rf)

Profit with Random Forest: 0
Optimal Threshold for Random Forest: 0.18084288357003442
Maximized Profit with Random Forest: 580


**Analysis**

According to the results, adjusting the threshold for the random forest model can maximize profits. The best threshold value for this model is 0.1812, which is lower than the value found for the logistic regression model. When using the logistic regression threshold for the random forest model, the profits are zero, indicating poor performance. Optimizing the threshold specifically for the random forest model increases profits to 580 euros.

The difference in optimal thresholds for these models can be attributed to their unique ways of capturing patterns in the data and scaling prediction probabilities. Thus, a threshold that works well for one model may not be effective for the other.

In conclusion, to maximize profits with the random forest model, the threshold should be lowered to 0.1812. Although the profits are lower than those achieved with the logistic regression model, they are still better than using the logistic regression threshold.